# Lesson 02
## Парсинг данных. HTML, DOM, XPath

Написать приложение и функцию, которые собирают основные новости с сайта на выбор dzen.ru, lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации

минимум один сайт максимум все 

In [1]:
from pprint import pprint
from lxml import html
import requests
import pandas as pd

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
URL_YANDES_NEWS = 'https://news.mail.ru/'
URL_LENTA_NEWS = 'https://lenta.ru/'
URL_DZEN_NEWS = 'https://dzen.ru/news/'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

params = {
    'sso_failed': '',
}

In [4]:
def get_content_dom_from_html_text(url, headers=None, params=None):
    response = requests.get(url, headers=headers, params=params)
    content_dom = html.fromstring(response.text)

    print(response.url)
    
    return content_dom


def parser_dzen_news(content_dom):
    news_container = content_dom.xpath("//div[contains(@class,'mg-card ')]")
    dzen_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//a[@class='mg-card__source-link']/text()")[0]
        new_text = new_container.xpath(".//h2/a/text()")[0].replace('\xa0', ' ')
        new_link = new_container.xpath(".//h2/a/@href")[0]
        new_date = new_container.xpath(".//span[@class='mg-card-source__time']/text()")[0]
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        dzen_news.append(news_dict) 

    return dzen_news


def parser_lenta_news(content_dom):
    news_container = content_dom.xpath("//a[contains(@class, 'card-')]")
    lenta_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//*[name()='svg']/*[name()='use']/attribute::*")
        if len(new_source) == 1:
            new_source = new_source[0].split('ui-label_')
            if len(new_source) == 2:
                new_source = new_source[1]
            else:
                new_source = 'Lenta.ru'
        else:
            new_source = 'Lenta.ru' 
        new_text = new_container.xpath(".//span[contains(@class, 'card-')]/text()")
        if len(new_text) == 1:
            new_text = new_text[0]
        else:
            new_text = None  
        new_link = URL_LENTA_NEWS + new_container.xpath("./@href")[0]
        
        new_date = new_container.xpath(".//time/text()")
        if len(new_date) == 1:
            new_date = new_date[0]
        else:
            new_date = None
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        lenta_news.append(news_dict)
        
    return lenta_news


def get_news(content_dom, parser_name='dzen'):
    result = None
    
    if parser_name == 'dzen':
      result = parser_dzen_news(content_dom)
    elif parser_name == 'yandex':
      result = parser_yandex_news(content_dom)
    elif parser_name == 'lenta':
      result = parser_lenta_news(content_dom)
    
    return result

In [5]:
dom = get_content_dom_from_html_text(URL_DZEN_NEWS, headers=headers, params=params)

https://dzen.ru/news?issue_tld=ru&sso_failed=


In [8]:
dzen_news = get_news(dom, 'dzen')
dzen_news[:2]

[{'new_source': 'Lenta.ru',
  'new_text': 'Мэр Енакиево Храменков сообщил, что над территорией города был сбит самолет',
  'new_link': 'https://dzen.ru/news/story/Mehr_Enakievo_KHramenkov_soobshhil_chto_nadterritoriej_goroda_byl_sbit_samolet--1476de7d07400648eeea2798723b3992?lang=ru&rubric=index&fan=1&stid=C2RIPTBj9kDsQXfIncH5&t=1677861386&tt=true&persistent_id=2253360001&story=022f378e-2255-5da2-8809-ebd282df41d5&issue_tld=ru',
  'new_date': '18:58'},
 {'new_source': 'РИА Новости',
  'new_text': 'Президент Путин подписал указ о работе оборонных предприятий во время военного положения',
  'new_link': 'https://dzen.ru/news/story/Prezident_Putin_podpisal_ukaz_orabote_oboronnykh_predpriyatij_vo_vremya_voennogo_polozheniya--287efa9b1dcf70ae7faa70c453885275?lang=ru&rubric=index&fan=1&stid=ao2RjwyEBv-3gTgqYaBO&t=1677861386&tt=true&persistent_id=2252961225&story=e17b91d2-d344-568f-a572-3134f0f4f9e4&issue_tld=ru',
  'new_date': '19:24'}]

In [9]:
pd.DataFrame(dzen_news)

,new_source,new_text,new_link,new_date
0,Lenta.ru,"Мэр Енакиево Храменков сообщил, что над террит...",https://dzen.ru/news/story/Mehr_Enakievo_KHram...,18:58
1,РИА Новости,Президент Путин подписал указ о работе оборонн...,https://dzen.ru/news/story/Prezident_Putin_pod...,19:24
2,Lenta.ru,СК возбудил уголовное дело по факту нападения ...,https://dzen.ru/news/story/SK_vozbudil_ugolovn...,19:23
3,РИА Новости,Conversation: ВСУ могут побояться развертывать...,https://dzen.ru/news/story/Conversation_VSU_mo...,19:33
4,РИА Новости,Прощание с журналистом и политологом Глебом Па...,https://dzen.ru/news/story/Proshhanie_szhurnal...,19:13
...,...,...,...,...
60,РИА Новости,«АвтоВАЗ» возобновил серийное производство Lad...,https://dzen.ru/news/story/AvtoVAZ_vozobnovil_...,19:32
61,Российская газета,В 23 городах России стартовали продажи автомоб...,https://dzen.ru/news/story/V23_gorodakh_Rossii...,18:19
62,Автоновости дня,Российские каршеринги начали массово распродав...,https://dzen.ru/news/story/Rossijskie_karsheri...,16:20
63,CarsWeek.ru,Нехватка водителей с Украины затруднила вывоз ...,https://dzen.ru/news/story/Nekhvatka_voditelej...,18:35


In [10]:
dom, response = get_content_dom_from_html_text(URL_LENTA_NEWS, headers=headers)

https://lenta.ru/


In [15]:
lenta_news=get_news(dom, 'lenta')
lenta_news[:3]

[{'new_source': 'Lenta.ru',
  'new_text': None,
  'new_link': 'https://lenta.ru//news/2023/03/03/netnet/',
  'new_date': '19:18'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Названы столичные районы с арендой квартир менее 30 тысяч рублей в месяц',
  'new_link': 'https://lenta.ru/https://moslenta.ru/news/city/s-arendoi-kvartir-03-03-2023.htm/',
  'new_date': '19:45'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Оценено падение доходов России от нефти и газа',
  'new_link': 'https://lenta.ru//news/2023/03/03/nedodohody/',
  'new_date': '19:44'}]

In [16]:
pd.DataFrame(lenta_news)

,new_source,new_text,new_link,new_date
0,Lenta.ru,None,https://lenta.ru//news/2023/03/03/netnet/,19:18
1,Lenta.ru,Названы столичные районы с арендой квартир мен...,https://lenta.ru/https://moslenta.ru/news/city...,19:45
2,Lenta.ru,Оценено падение доходов России от нефти и газа,https://lenta.ru//news/2023/03/03/nedodohody/,19:44
3,Lenta.ru,В сети обругали Эмили Ратаковски из-за цветка ...,https://lenta.ru//news/2023/03/03/wearsplant/,19:43
4,Lenta.ru,Трое россиян привязали должника к машине и про...,https://lenta.ru//news/2023/03/03/pokhishenie/,19:37
...,...,...,...,...
136,Lenta.ru,Как они изменят жизнь студентов и регионов?,https://lenta.ru//articles/2023/02/28/studgrd/,"17:05, 28 февраля 2023"
137,Lenta.ru,Президент Латвии подписал декларацию о поддерж...,https://lenta.ru//news/2023/03/03/ukr_laatvia/,16:54
138,Lenta.ru,"Диссидент, идеолог Кремля, «дурачок для битья...",https://lenta.ru//articles/2023/02/27/glebpavl...,"18:43, 27 февраля 2023"
139,Lenta.ru,Россия усилит информационно-аналитическое взаи...,https://lenta.ru//news/2023/03/03/anltreg/,16:54
